The following data is generated from a separate Python project called CONTROL, in which teams play against one another in a fictional sport based on RNG. 8 groups of 100 play 10 matches against everyone within their group, keeping track of their wins and losses, for matches and lineups. (A match is made up of 8 Games between 8 different lineups. A Lineup Win/Loss is given for each of these Games, a Match Win/Loss is given if one team wins at least 5 Games, and a Match Draw is given to both teams if each team wins 4 Games. Regular season seeding (which is all we are concerned with at the moment) is based on Lineup win/loss rate.)

Teams have 6 total players who play in lineups of 4, with the best players playing more often.
Each statistic NOT including wins/losses (Power, Attack Damage, etc.) is calculated as a weighted average of all of the team's players.
Out of a total of 9 lineups, the best player plays in all 9, so their stats are weighted accordingly, as with the others based on the proportion of lineups where they are in/out.

In [165]:
import pandas as pd
import numpy as np

#Read DataFrame from Excel file generated from PyCharm project
team_stats = pd.read_excel("C:/Users/carte/OneDrive/Documents/ControlAverageStats.xlsx")
team_stats['Lineup Win %'] = (team_stats['Lineup Wins'] / ((team_stats['Lineup Losses']) + team_stats['Lineup Wins'])).round(4)

team_stats['Match Win %'] = (team_stats['Match Wins']*2 + team_stats['Match Draws']) / 
((team_stats['Match Losses'] + team_stats['Match Wins'] + team_stats['Match Draws'])*2).round(3)

#Check DataFrame to make sure it exists
team_stats.head()

,Team,Power,Attack Damage,Attack Speed,Critical %,Critical X,Health,Spawn Time,Lineup Wins,Lineup Losses,Match Wins,Match Losses,Match Draws,Lineup Win %,Match Win %
0,Darkwing_Vampires,55.67,51.00,9.44,0.03744,9.647,555.102,11.89,6011,1909,880,35,75,0.7590,0.926768
1,Darkwing_Voidwalkers,56.28,54.39,10.61,0.03772,10.703,553.165,12.06,5836,2084,847,48,95,0.7369,0.903535
2,Darkwing_Centaur,55.61,50.06,9.75,0.04144,10.233,546.369,13.17,5579,2341,802,66,122,0.7044,0.871717
3,Darkwing_Wings,54.08,52.00,8.97,0.03583,10.203,557.721,12.75,5546,2374,781,84,125,0.7003,0.852020
4,Darkwing_Manticore,56.31,49.06,9.97,0.03617,9.493,541.391,11.25,5539,2381,756,79,155,0.6994,0.841919


Now, let's analyze this data. I am primarily concerned with Lineup Win %, and how each of the player statistics impact this.
There are a number of ways we can go about this, but let's begin with a Random Forest Regression and see where that gets us.
From my own experience, Power is by far the most important metric, so let's see if the model agrees.

In [166]:
#Import necessary modules
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler

In [167]:
team_stats_copy = team_stats.copy()

#We only want statistics related to the players, so let's get rid of everything else for the features
features = team_stats_copy.drop(columns=['Team', 'Lineup Wins', 'Lineup Losses',
                                         'Match Wins', 'Match Losses', 'Match Draws',
                                         'Lineup Win %', 'Match Win %'])

#Lineup Win% determines regular season seeding, so it is my target
target = team_stats['Lineup Win %']

#Because the ranges for each statistic are very different, we should normalize them first
scaler = MinMaxScaler()

features_scaled = scaler.fit_transform(features)
features_scaled = pd.DataFrame(features_scaled, columns=features.columns, index=features.index)

#Split data into training and testing. With 800 samples, 150 is 18.75%, an adequate amount for testing
features_train, features_test, target_train, target_test = train_test_split(features_scaled, target,
                                                                            test_size=150, random_state=42)

In [168]:
#Create and fit the model!
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(features_train, target_train)

RandomForestRegressor(random_state=42)

In [169]:
#Now, let's find the importance of each feature in predicting the target
feature_importances = model.feature_importances_
for name, importance in zip(features_scaled.columns, feature_importances):
    print(f"{name}: {importance}")

Power: 0.25733369545206386
Attack Damage: 0.09520366299505004
Attack Speed: 0.526266916524278
Critical %: 0.034310615329579255
Critical X: 0.019742442928670667
Health: 0.020037267966050005
Spawn Time: 0.04710539880430817


Most of this makes sense, although I did not expect Attack Speed to be so influential, I also expected Power to be above 50%.
This is only one model though, perhaps others will produce different numbers.

In [222]:
#Predict the number of wins for a team with 60 power, 45 Attack Damage, 9 Attack Speed, 0.045Crit%, 11CritX, 540 Health, 11Spawn
theory_team = pd.DataFrame([{'Power' : 60, 'Attack Damage' : 45, 'Attack Speed' : 9, 'Critical %' : 0.045, 'Critical X' : 11,
                           'Health' : 540, 'Spawn Time' : 10.7}])
#From my experience, a team with these statistics would destroy the competition
theory_team_scaled = scaler.transform(theory_team)
theory_team_scaled = pd.DataFrame(theory_team_scaled, columns=theory_team.columns)

print(f"Predicted Lineup Win %: {float(model.predict(theory_team_scaled)):.3f}")

Predicted Lineup Win %: 0.678


In [171]:
#Let's see how that team compares to the 800 test samples by sorting them
teams_sorted = team_stats.sort_values(by='Lineup Win %', ascending=False)
teams_sorted.head()

,Team,Power,Attack Damage,Attack Speed,Critical %,Critical X,Health,Spawn Time,Lineup Wins,Lineup Losses,Match Wins,Match Losses,Match Draws,Lineup Win %,Match Win %
400,Ice-Wall_Sirens,56.14,50.64,8.83,0.03986,10.295,553.473,11.67,6279,1641,910,13,67,0.7928,0.953030
700,Steel-Heart_Revolution,56.19,51.31,8.31,0.03722,10.959,538.521,13.75,6157,1763,893,31,66,0.7774,0.935354
600,Hell's-Circle_Watchers,56.75,50.89,11.00,0.04122,10.227,552.484,12.64,6034,1886,884,31,75,0.7619,0.930808
300,Web-of-Nations_Exorcists,56.67,50.44,10.22,0.04017,10.085,549.021,11.75,6029,1891,849,38,103,0.7612,0.909596
0,Darkwing_Vampires,55.67,51.00,9.44,0.03744,9.647,555.102,11.89,6011,1909,880,35,75,0.7590,0.926768


In order to make sure that Power truly is not as valuable as I thought, I need to put this to the test by inserting a real team with the same statistics as theory_team into the mix and see how they do.
This will be difficult for a number of reasons: first, I don't want to run all 800 teams again, since that was a painstaking process in the first place. Second, these values are weighted averages from all players, so there are many different team combinations which could have stats identical to theory_team.

The best way to get around both of these issues is to try 6 different groups of 100 teams, each including 5  or 6 different possible iterations of theory_team. In one of these iterations, all of the players will have identical values equal to the theory_team weighted averages. I suspect this team will win the overwhelming majority of its Games, upwards of 80%.
For all of the others, I will create functions which create random values that have a weighted average for each statistic equal to that of the theory_team within constraints reasonable to the game. The code in which I create these values and assign them to my model teams is shown below. It can't run, as this notebook doesn't have access to the project for the Team and Player classes, but it's here nonetheless.

    def generate_list(target_total, min_value, max_value, round_to=0):
        final_list = []

        coefficients = np.array([9, 7, 6, 6, 4, 4])

        num_lists = 1

        valid_list_found = False

        while not valid_list_found:

            if round_to == 0:
                initial_values = np.random.randint(min_value, max_value + 1, size=5)
            else:
                initial_values = np.round(np.random.uniform(min_value, max_value, size=5), round_to)

            partial_sum = np.dot(coefficients[:-1], initial_values)
            remaining_value = (target_total - partial_sum) / coefficients[-1]

            if min_value <= remaining_value <= max_value:
                valid_list_found = True

                final_list = np.append(initial_values, remaining_value)


        return final_list

    #model_teams = create_teams(6, 'Test')
    model_teams[0].name = "Test_CLONES"
    for player in model_teams[0].players:
        player.power  = 60
        player.atk_dmg = 45
        player.atk_spd = 9
        player.crit_pct = 0.045
        player.crit_x = 10.7
        player.max_health = 540
        player.spawn_time = 11

    total_power_target = 2160 #round to 0
    total_atk_dmg_target = 1620 #round to 0
    total_atk_spd_target = 324 #round to 0
    total_crit_pct_target = 1.62 #round to 4
    total_crit_x_target = 385 #round to 2
    total_health_target = 19440 #round to 2
    total_spawn_target = 396 #round to 0


    for i in range(1,6): #create model teams 1 through 5
        idx = 0
        gen_power_list = generate_list(total_power_target, 47, 63, 0)
        gen_atk_dmg_list = generate_list(total_atk_dmg_target, 35, 65, 0)
        gen_atk_spd_list = generate_list(total_atk_spd_target, 6, 14, 0)
        gen_crit_pct_list = generate_list(total_crit_pct_target, 0.02, 0.06, 4)
        gen_crit_x_list = generate_list(total_crit_x_target, 6, 12, 2)
        gen_health_list = generate_list(total_health_target, 500, 675, 2)
        gen_spawn_list = generate_list(total_spawn_target, 9, 15, 0)
        for player in model_teams[i].players:
            player.power = gen_power_list[idx]
            player.atk_dmg = gen_atk_dmg_list[idx]
            player.atk_spd = gen_atk_spd_list[idx]
            player.crit_pct = gen_crit_pct_list[idx]
            player.crit_x = gen_crit_x_list[idx]
            player.max_health = gen_health_list[idx]
            player.spawn_time = gen_spawn_list[idx]
            idx += 1

In [194]:
#Now, I have 30 teams with weighted averages identical to theory_team, and I suspect that these teams will dominate
#which would indicate a flaw in the model.
#However, if the 30 teams collectively win around 67.8% of their games, which is what the model predicted theory_team would,
#then the model looks very good and needs more testing to confirm.
#One thing to consider is the radical difference between teams with the same weighted averages, which could lead to
#differing degrees of success among the teams. This is the primary reason I chose to create so many iterations of theory_team.

#I will run the code in PyCharm, and we can take a look at the results from Excel.
team_stats = pd.read_excel("C:/Users/carte/OneDrive/Documents/ControlAverageStats.xlsx")
team_stats['Lineup Win %'] = (team_stats['Lineup Wins'] / ((team_stats['Lineup Losses']) + team_stats['Lineup Wins'])).round(4)

team_stats['Match Win %'] = (team_stats['Match Wins']*2 + team_stats['Match Draws']) / 
((team_stats['Match Losses'] + team_stats['Match Wins'] + team_stats['Match Draws'])*2).round(3)

teams_sorted = team_stats.sort_values(by='Lineup Win %', ascending=False)
teams_sorted.head(10)

#All teams starting with "Test_" are the model teams with weighted averages equal to theory_team
#Test_CLONES is the team with all players having the exact same stats, those being equal to the weighted averages.

,Team,Power,Attack Damage,Attack Speed,Critical %,Critical X,Health,Spawn Time,Lineup Wins,Lineup Losses,Match Wins,Match Losses,Match Draws,Lineup Win %,Match Win %
210,Test_Necromancers,60.0,45.0,9.0,0.045,10.722,540.0,11.0,2250,246,306,2,4,0.9014,0.987179
421,Test_Sprites,60.0,45.0,9.0,0.045,10.722,540.0,11.0,2208,288,305,2,5,0.8846,0.985577
420,Test_Zombies,60.0,45.0,9.0,0.045,10.722,540.0,11.0,2208,288,304,5,3,0.8846,0.979167
525,Test_Druids,60.0,45.0,9.0,0.045,10.722,540.0,11.0,2198,298,302,3,7,0.8806,0.979167
526,Test_Elementals,60.0,45.0,9.0,0.045,10.722,540.0,11.0,2197,299,306,2,4,0.8802,0.987179
105,Test_Illusionists,60.0,45.0,9.0,0.045,10.722,540.0,11.0,2186,310,306,1,5,0.8758,0.988782
527,Test_Acolytes,60.0,45.0,9.0,0.045,10.722,540.0,11.0,2182,314,302,4,6,0.8742,0.977564
422,Test_Harpy,60.0,45.0,9.0,0.045,10.722,540.0,11.0,2179,317,304,1,7,0.8730,0.985577
0,Test_Minotaurs,60.0,45.0,9.0,0.045,10.722,540.0,11.0,2174,322,308,0,4,0.8710,0.993590
528,Test_Wraiths,60.0,45.0,9.0,0.045,10.722,540.0,11.0,2144,352,304,4,4,0.8590,0.980769


In [212]:
#Let's calculate the total Lineup Win% for all Test teams to see how reality stacks up to the model
test_teams_results = teams_sorted[teams_sorted['Team'].str.contains('Test_')]
print(f"Teams with weighted averages of 60 Power, 45 Attack Speed, \
0.045 Critical Chance,\n10.7 Critical Multiplier, \
540 Health and 11 Spawn Time win an average of \
{100*np.mean(test_teams_results['Lineup Win %'])}% of their Lineups.")

Teams with weighted averages of 60 Power, 45 Attack Speed, 0.045 Critical Chance,
10.7 Critical Multiplier, 540 Health and 11 Spawn Time win an average of 64.946% of their Lineups.


In [221]:
#According to our data, teams with stats equal to theory_team win 64.946% of their games.
#Finally, let's do a binomial test to see if this is significantly different. Then, finally, we can determine
#if our model is flawed or not.
from scipy.stats import binomtest

p_predicted = 0.678 #value predicted by model
observed_wins = np.sum(test_teams_results['Lineup Wins'])
observed_losses = np.sum(test_teams_results['Lineup Losses'])

n = observed_wins + observed_losses

result = binomtest(observed_wins, n, p_predicted)
print(f"The P-Value is {result.pvalue}.")

The P-Value is 8.719613675105577e-62.


Considering this value is virtually 0, we can conclude with near-certainty that the RandomForestRegression model created to predict winrate based on weighted averages of player stats is in some way flawed, so I will need to search for a better model to achieve better accuracy.